In [1]:
# Notebook parameters. Values here are for development only and
# will be overridden when running via snakemake and papermill.
country = "ML"
contigs = ["3L"]
analysis_version = "2025.02.13"
dask_scheduler = "single-threaded"
cohorts_analysis = "20240924"

In [2]:
# Parameters
analysis_version = "2025.02.25"
min_cohort_size = 15
max_cohort_size = 100
sample_sets = [
    "3.0",
    "1237-VO-BJ-DJOGBENOU-VMF00050",
    "1237-VO-BJ-DJOGBENOU-VMF00067",
    "1244-VO-GH-YAWSON-VMF00051",
    "1245-VO-CI-CONSTANT-VMF00054",
    "1253-VO-TG-DJOGBENOU-VMF00052",
    "1178-VO-UG-LAWNICZAK-VMF00025",
    "1244-VO-GH-YAWSON-VMF00149",
    "barron-2019",
    "crawford-2016",
    "tennessen-2021",
    "bergey-2019",
    "campos-2021",
    "fontaine-2015-rebuild",
]
sample_query = "taxon in ['gambiae', 'coluzzii', 'arabiensis', 'bissau']"
contigs = ["2RL", "3RL", "X"]
cohorts_analysis = "20240924"
h12_calibration_contig = "3L"
h12_signal_detection_min_delta_aic = 1000
h12_signal_detection_min_stat_max = 0.1
h12_signal_detection_gflanks = [6]
dask_scheduler = "single-threaded"
alerts = ["01", "02", "03", "04", "05", "06", "07", "08", "09"]
country = "GH"


In [3]:
from pyprojroot import here

root = here()
%run -i {root}/workflow/common/scripts/setup.py
%run -i {root}/workflow/site/scripts/page-setup.py

# N.B., do not add the "remove-output" tag to this cell!!! If you do,
# the bokeh javascript libraries will not get loaded in the generated
# HTML page. The call to output_notebook() injects javascript in the
# cell output which triggers the bokeh javascript libraries to be loaded
# in the page.
output_notebook(hide_banner=True)

# Ghana

The map below shows geographical regions where mosquito genomic data are available and {term}`genome-wide selection scans` have been performed.

Click on a map marker to see more information about mosquito {term}`cohorts` for which results are available.

In [4]:
gdf_cohorts_country = gdf_cohorts.query(f"country_alpha2 == '{country}'")

In [5]:
plot_cohorts_map(gdf_cohorts=gdf_cohorts_country, zoom=6, url_prefix="../")

Map(center=[5.913362832682055, -1.400973103486698], controls=(ZoomControl(options=['position', 'zoom_in_text',…

<br></br>

In [6]:
def make_clickable(url, name):
    return '<a href="{}" rel="noopener noreferrer" target="_blank">{}</a>'.format(
        url, name
    )

In [7]:
gdf_cohorts = gdf_cohorts_country.reset_index()
gdf_cohorts["url"] = "../cohort/" + gdf_cohorts["cohort_id"] + ".html"
gdf_cohorts["cohort_label"] = gdf_cohorts.apply(
    lambda x: make_clickable(x["url"], x["cohort_label"]), axis=1
)

gdf_cohorts = gdf_cohorts[
    [
        "cohort_label",
        "admin1_name",
        "admin2_name",
        "taxon",
        "year",
        "quarter",
        "cohort_size",
    ]
]
gdf_cohorts = gdf_cohorts.rename(
    columns={
        "cohort_label": "Cohort",
    }
).set_index("Cohort")
gdf_cohorts.columns = [
    "Region",
    "District",
    "Taxon",
    "Year",
    "Quarter",
    "Sample Size",
]
gdf_cohorts.style

,Region,District,Taxon,Year,Quarter,Sample Size
Cohort,,,,,,
Ghana / Ablekuma Central Municipal / coluzzii / 2018 / Q1,Greater Accra Region,Ablekuma Central Municipal,coluzzii,2018,1,266
Ghana / La-Nkwantanang-Madina / gambiae / 2017 / Q4,Greater Accra Region,La-Nkwantanang-Madina,gambiae,2017,4,200
Ghana / Adansi Akrofuom / coluzzii / 2018 / Q4,Ashanti Region,Adansi Akrofuom,coluzzii,2018,4,64
Ghana / Adansi South / coluzzii / 2018 / Q4,Ashanti Region,Adansi South,coluzzii,2018,4,36
Ghana / Adansi South / gambiae / 2018 / Q4,Ashanti Region,Adansi South,gambiae,2018,4,29
Ghana / Amansie Central / coluzzii / 2018 / Q4,Ashanti Region,Amansie Central,coluzzii,2018,4,69
Ghana / Bekwai Municipal / coluzzii / 2018 / Q4,Ashanti Region,Bekwai Municipal,coluzzii,2018,4,53
Ghana / Twifo Atti-Morkwa / coluzzii / 2012 / Q3,Central Region,Twifo Atti-Morkwa,coluzzii,2012,3,25
Ghana / Upper Denkyira East Municipal / coluzzii / 2018 / Q4,Central Region,Upper Denkyira East Municipal,coluzzii,2018,4,23
